In [ ]:
from nbdev import *
%nbdev_default_export data_man
%reload_ext autoreload
%autoreload 2

Cells will be exported to wtlike.data_man,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# Data management
> Create, from FT1 and FT2, a compact data set with photon and livetime info.

### Overview

Fermi-LAT data files are extracted from the FTP server 
`https://heasarc.gsfc.nasa.gov/FTP/fermi/data/lat/weekly`, with subfolders for the photon data, `photon` and spacecraft data, `spacecraft`. It is described `http://fermi.gsfc.nasa.gov/ssc/data/access/http://fermi.gsfc.nasa.gov/ssc/data/access/`

The class `WeeklyData` downloads these to temporary files and constructs a dict for each week with
contents

* tstart: reference time
* photons: a table, entry per selected photon with columns 
  * time since tstart
  * energy and event type
  * position as HEALPix index
* sc_data: a table, an entry per 30-s interval, with columns
  * start/stop time
  * S/C direction
* gti_times: an array of interleaved start/stop intervals

These dict objects, one per week, are saved in a folder.


In [ ]:
%nbdev_export
from astropy.io import fits

import healpy
from pathlib import Path
import numpy as np
import pandas as pd
pd.set_option('display.precision', 2)
import pickle
from wtlike.config import *

In [ ]:
%nbdev_export
def get_ft1_data( config, ft1_file):

        """
        Read in a photon data (FT1) file, bin in energy and position to convert to a compact DataFrame
        
        - `ft1_file` -- A monthly file generated by J. Ballet, or a weekly file from GSFC
        
        Depends on config items
        - `theta_cut, z_cut` -- selection criteria
        - `ebins, etypes` -- define band index
        - `nside, nest` -- define HEALPix binning
         
        Returns a tuple with 

        - `tstart`, the start MET time

        - DataFrame  with columns
           - `band` (uint8):    energy band index*2 + 0,1 for Front/Back 
           - `nest_index`  if nest else `ring_index` (uint32): HEALPIx index for the nside 
           - `time` (float32):    the elapsed time in s from header value TSTART in the FT1 file
           
        - gti times as an interleaved start, stop array.  
        
        For the selected events above 100 MeV, this represents 9 bytes per photon, vs. 27.
        """
      
        ebins = config.energy_edges
        etypes = config.etypes
        nside = config.nside
        nest = config.nest
        z_cut =config.z_max
        theta_cut = np.degrees(np.arccos(config.cos_theta_max))
        verbose = config.verbose
        
        with  fits.open(ft1_file) as ft1:
            tstart = ft1[1].header['TSTART'] 

            ## GTI - setup raveled array function to make cut
            gti_data= ft1['GTI'].data
            # extract arrays for values of interest
            data =ft1['EVENTS'].data

        a,b = sorted(gti_data.START), sorted(gti_data.STOP)
        
        gti_times = np.ravel(np.column_stack((a,b)))
        if np.any(np.diff(gti_times)<0):
            raise Exception(f'Non-monatonic GTI found')
            
        def apply_gti(time):
            x = np.digitize(time, gti_times)
            return np.bitwise_and(x,1).astype(bool)
        if verbose>1:
            total = sum(b)-sum(a)
            fraction = total/(b[-1]-a[0])
  

        glon, glat, energy, et, z, theta, time, ec =\
             [data[x] for x in 'L B ENERGY EVENT_TYPE ZENITH_ANGLE THETA TIME EVENT_CLASS'.split()]
        
        # generate event_type masks
        et_mask={}
        for ie in etypes:
            et_mask[ie]= et[:,-1-ie]
            

            
        data_cut = np.logical_and(theta<theta_cut, z<z_cut)
        e_cut = energy>ebins[0]
        gti_cut = apply_gti(time)
        
        if verbose>1:
            total = sum(b)-sum(a)
            fraction = total/(b[-1]-a[0])
  
            print(  f'FT1: {ft1_file.name}, GTI range {a[0]:.1f}-{b[-1]:.1f}'\
                    f'\n\t {len(data):,} photons, {sum(e_cut):,} with E>{ebins[0]:.0f} MeV.'\
                    f'\n\ttheta<{theta_cut:.1f} and z<{z_cut} selections remove:'\
                    f' {100.- 100*sum(data_cut)/float(len(data)):.2f}%'\
                    f', GTI cut removes {sum(~gti_cut)}'
                 )
        # apply selection
        sel = e_cut & data_cut & gti_cut
        glon_sel = glon[sel]
        glat_sel = glat[sel]
        
        # event class -- turn into single int for later mask
#         bits = np.array([1<<n for n in range(20)])
#         def to_bin(x):
#             return np.sum(bits[x[:20]])
#         ec = [to_bin(row[20]) for row in ec[sel]]
        
        # pixelate direction
        hpindex = healpy.ang2pix(nside, glon_sel, glat_sel, nest=nest, lonlat=True).astype(np.int32)
        hpname = 'nest_index' if nest else 'ring_index'
        
        # digitize energy and create band index incluing (front/back)
        ee = energy[sel]
        band_index = (2*(np.digitize(ee, ebins, )-1) + et_mask[1][sel]).astype(np.uint8)

        # combine into a recarray to feed to pandas
        recarray = np.rec.fromarrays(
                    [ band_index,  
                     hpindex,  
                     (time-tstart)[sel].astype(np.float32),
                    # ec, ### temp
                    ], 
                    names=['band', hpname, 'time',
                           #'ec', #### temp
                          ])
        if verbose>1:
            print(f'\tReturning tstart={tstart:.0f}, {len(recarray):,} photons.')
            
        return  tstart, pd.DataFrame.from_records(recarray), gti_times

In [ ]:
show_doc(get_ft1_data, title_level=2)

<h2 id="get_ft1_data" class="doc_header"><code>get_ft1_data</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h2>

> <code>get_ft1_data</code>(**`config`**, **`ft1_file`**)

Read in a photon data (FT1) file, bin in energy and position to convert to a compact DataFrame

- `ft1_file` -- A monthly file generated by J. Ballet, or a weekly file from GSFC

Depends on config items
- `theta_cut, z_cut` -- selection criteria
- `ebins, etypes` -- define band index
- `nside, nest` -- define HEALPix binning
 
Returns a tuple with 

- `tstart`, the start MET time

- DataFrame  with columns
   - `band` (uint8):    energy band index*2 + 0,1 for Front/Back 
   - `nest_index`  if nest else `ring_index` (uint32): HEALPIx index for the nside 
   - `time` (float32):    the elapsed time in s from header value TSTART in the FT1 file
   
- gti times as an interleaved start, stop array.  

For the selected events above 100 MeV, this represents 9 bytes per photon, vs. 27.

In [ ]:
%nbdev_export
def get_ft2_info(config, filename, 
                 gti=lambda t: True):
    """Process a FT2 file, with S/C history data, and return a summary DataFrame
    
    Parameters:

    * config -- verbose, cos_theta_max, z_max        
    * filename -- spacecraft (FT2) file       
    * gti -- GTI object that checkes for allowed intervals, in MJD units

    Returns: A DataFrame with fields consistent with GTI if specified 
        
    * start, stop -- interval in MJD units
    * livetime -- sec
    * ra_scz, dec_scz --spaceraft direction
    * ra_zenith, dec_zenith -- local zenith
    """
    # combine the files into a DataFrame with following fields besides START and STOP (lower case for column)
    fields    = ['LIVETIME','RA_SCZ','DEC_SCZ', 'RA_ZENITH','DEC_ZENITH']
    with fits.open(filename) as hdu:
        scdata = hdu['SC_DATA'].data
        tstart, tstop = [float(hdu[0].header[field]) for field in  ('TSTART','TSTOP') ]
    
    if config.verbose>1:
        print(f'FT2: {filename.name}, MET range {tstart:.1f}-{tstop:.1f},', end='')# {"not" if gti is None else ""} applying GTI')
        
    # get times to check against MJD limits and GTI
    start, stop = [MJD(np.array(scdata.START, float)),
                   MJD(np.array(scdata.STOP, float))]
    
    # apply GTI to bin center (avoid edge effects?)
    in_gti = gti(0.5*(start+stop)) 
    if config.verbose>1:
        s = sum(in_gti)
        print(f' {len(start)} entries, {s} ({100*s/len(start):.1f}%) in GTI')
    
    t = [('start', start[in_gti]), ('stop',stop[in_gti])]+\
        [(field.lower(), np.array(scdata[field][in_gti],np.float32)) for field in fields ]

    sc_data = pd.DataFrame(dict(t) ) 
            
    return sc_data

In [ ]:
show_doc(get_ft2_info, title_level=2)

<h2 id="get_ft2_info" class="doc_header"><code>get_ft2_info</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h2>

> <code>get_ft2_info</code>(**`config`**, **`filename`**, **`gti`**=*`None`*)

Process a FT2 file, with S/C history data, and return a summary DataFrame

Parameters:

* config -- verbose, cos_theta_max, z_max        
* filename -- spacecraft (FT2) file       
* gti -- GTI object with allowed intervals or None

Returns: A DataFrame with fields 
    
* start, stop -- interval in MJD units
* livetime -- sec
* ra_scz, dec_scz --spaceraft direction
* ra_zenith, dec_zenith -- local zenith

In [ ]:
%nbdev_hide
filename ='/tmp/from_gsfc/week668_ft2.fits'
if os.path.exists(filename):
    config = Config()
    config.verbose=3
    t = get_ft2_info(config, filename); 
    print(t.head())
else:
    print(f'File {filename} does not exist, skipping get_ft2_info test')

File /tmp/from_gsfc/week668_ft2.fits does not exist, skipping get_ft2_info test


In [ ]:
%nbdev_export
class WeeklyData(object):
    """Download and process weekly Fermi-LAT files from GSFC,
    
    at FTP 'https://heasarc.gsfc.nasa.gov/FTP/fermi/data/lat/weekly' 
    
    * week: a mission week number
    * saveto: path to save the files
    
    Creates a pickled file as  dict with
    
    * tstart--start time (MJD)
    * photons -- DataFrame with photon data--see `get_ft1_data`
    * sc_data -- DataFrame with Spacecraft data--see `get_ft2_info` 
    * gti_times -- array of interleaved start/stop times from the photon data 
    """
    
    ftp = 'https://heasarc.gsfc.nasa.gov/FTP/fermi/data/lat/weekly'
    tmp = Path('/tmp/from_gsfc')
    
    def __init__(self, config, week, saveto):
        """
        * week: a mission week number, starting at 9
        * saveto: path to save the fiels
        
        """
        import wget
        self.config= config
        self.saveto=Path(saveto)
        os.makedirs(self.saveto, exist_ok=True)
        assert week>8
        self.wk = week
  
        self.fits_files = [self.tmp/f'week{week:03d}_{x}.fits' for x in 'ft1 ft2'.split()]
        os.makedirs(self.tmp, exist_ok=True)
        
        urls = []
        for ftype in  ['photon', 'spacecraft']:
             urls.append(f'{self.ftp}/{ftype}/lat_{ftype}_weekly_w{week:03d}_p{"305" if ftype=="photon" else "202" }_v001.fits')
        
        for url, fname in zip(urls, self.fits_files):
            if not fname.exists():
                if config.verbose>1:
                    print(f'{url.split("/")[-1]} -> {fname}')
                wget.download(str(url), str(fname))
            else: 
                if config.verbose>1: print(f'{fname} exists')

    def process_ft1(self):
        self.tstart, self.photon_data, self.gti_times = get_ft1_data(self.config, self.fits_files[0])
    
    def process_ft2(self):
        
        def apply_gti(time): # note MJD
            x = np.digitize(time, MJD(self.gti_times))
            return np.bitwise_and(x,1).astype(bool)
        
        self.sc_data = get_ft2_info(self.config, str(self.fits_files[1]), apply_gti)
        
    def save(self):
        """process, then save aa dict"""
        
        self.process_ft1()
        self.process_ft2()
        
        d = dict(tstart = self.tstart,
                photons = self.photon_data,
                sc_data = self.sc_data,
                gti_times = self.gti_times)
        filename = self.saveto/f'week_{self.wk:03d}.pkl'
        pickle.dump(d, open(filename, 'wb'))
        if self.config.verbose>0:
            print(f'Saved to {filename}')
        

In [ ]:
show_doc(WeeklyData)

<h2 id="WeeklyData" class="doc_header"><code>class</code> <code>WeeklyData</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>WeeklyData</code>(**`config`**, **`week`**, **`saveto`**)

Download and process weekly Fermi-LAT files from GSFC,

at FTP 'https://heasarc.gsfc.nasa.gov/FTP/fermi/data/lat/weekly' 

* week: a mission week number
* saveto: path to save the files

Creates a pickled file as  dict with

* tstart--start time (MJD)
* photons -- DataFrame with photon data--see [`get_ft1_data`](wtlike/data_man#get_ft1_data)
* sc_data -- DataFrame with Spacecraft data--see [`get_ft2_info`](wtlike/data_man#get_ft2_info) 
* gti_times -- array of interleaved start/stop times from the photon data 

In [ ]:
%nbdev_export
class GTI(object):
    
    def __init__(self, gti_times):
        self.mjd_gti = MJD(gti_times)
    
    def __call__(self, time:'float'):
        x = np.digitize(time, self.mjd_gti)
        return np.bitwise_and(x,1).astype(bool)

In [ ]:
%nbdev_export
class MonthlyData(object):
    """
    """
    monthly_path ='/mnt/c/users/thbur/tmp/monthly'
    ft1_pattern = 'ft2_20*.fits'
    ft2_pattern = 'P305_Source_*_zmax105.fits'
    
    def __init__(self, config, saveto):
        """
        """
        from pathlib import Path
        self.config=config
        folder = Path(self.monthly_path)
        assert folder.is_dir, f'Monthy folder {self.montly_path} not a folder'
        
        self.saveto=Path(saveto)
        assert self.saveto.is_dir, f'Output folder {self.saveto} is not a folder.'
        os.makedirs(saveto, exist_ok=True)
        self.ft2_files = sorted(folder.glob(self.ft1_pattern))
        self.ft1_files = sorted(folder.glob(self.ft2_pattern))
        print(f'MonthlyData initialized with {len(self.ft1_files)} monthly FT1'\
              f' and {len(self.ft2_files)} yearly FT2 files '\
             f'\n Saving to {saveto}')

    def process(self, month):
        """
        month: month index (1..12)
        """
        m = month-1
        y = (m+7)//12 # wired-in: data started in August 2008
        f1, f2 = self.ft1_files[m], self.ft2_files[y]
        assert f1.is_file(), f'FT1 file {f1} not found'
        assert f2.is_file(), f'FT2 files {f2} not found'
        tstart, p_df, gti_times = get_ft1_data(config, f1) 
        ft2_df = get_ft2_info(config, f2, gti=GTI(gti_times))
        d = dict(tstart = tstart,
            photons = p_df,
            sc_data = ft2_df,
            gti_times = gti_times)
        
        filename = self.saveto/f'month_{month:04d}.pkl'
        pickle.dump(d, open(filename, 'wb'))
        if self.config.verbose>0:
            print(f'Saved to {filename.name}')
        return d

## Develop montly

In [ ]:
md = MonthlyData(config, saveto='/home/burnett/monthly')


MonthlyData initialized with 152 monthly FT1 and 14 yearly FT2 files 
 Saving to /home/burnett/monthly


In [ ]:
%%capture cap
for m in range(1,152 ):
    md.process(m)

In [ ]:
md.process(152)

FT1: P305_Source_152_zmax105.fits, GTI range 636460000.0-639090000.0
	 2,784,890 photons, 1,727,130 with E>100 MeV.
	theta<66.4 and z<100 selections remove: 12.55%, GTI cut removes 0
	Returning tstart=636460000, 1,539,940 photons.
FT2: ft2_2021.fits, MET range 631200000.0-639090000.0, 222543 entries, 74146 (33.3%) in GTI
Saved to month_0152.pkl


{'tstart': 636460000.0,
 'photons':          band  nest_index      time
 0           3     7683316  1.89e+00
 1           4     8978568  3.73e+00
 2           3    12181250  8.77e+00
 3           0    12358109  8.99e+00
 4           0     7871599  9.46e+00
 ...       ...         ...       ...
 1539935     0    11878318  2.63e+06
 1539936     3    11979221  2.63e+06
 1539937     0     4632487  2.63e+06
 1539938     4     4897092  2.63e+06
 1539939     1     7400784  2.63e+06
 
 [1539940 rows x 3 columns],
 'sc_data':           start      stop  livetime  ra_scz  dec_scz  ra_zenith  dec_zenith
 0      59276.44  59276.44     25.13   45.93   -75.65      45.72      -25.65
 1      59276.44  59276.44     25.10   50.82   -75.62      47.81      -25.64
 2      59276.44  59276.44     25.14   55.67   -75.52      49.90      -25.60
 3      59276.44  59276.44     25.21   60.45   -75.35      51.99      -25.53
 4      59276.44  59276.44     25.23   65.12   -75.11      54.07      -25.43
 ...         ... 

In [ ]:
from pathlib import Path
folder = Path('/mnt/c/users/thbur/tmp/monthly')
ft2 = sorted(folder.glob('ft2_20*.fits')); ft2
ft1 = sorted(folder.glob('P305_Source_*_zmax105.fits'))
len(ft2), len(ft1)

(4, 94)

In [ ]:
%%time
dd = []
for m in range(12):
    dd.append( process(m))

FT1: P305_Source_001_zmax105.fits, GTI range 239557417.5-241960000.0
	 2,183,932 photons, 1,501,624 with E>100 MeV.
	theta<66.4 and z<100 selections remove: 4.40 %
	GTI cut removes 0
	Returning tstart=239557417, 1,418,664 photons.
FT2: ft2_2008.fits, MET range 239556977.6-252480028.6, 362996 entries, 66048 (18.2%) in GTI
FT1: P305_Source_002_zmax105.fits, GTI range 241960000.0-244590000.0
	 2,370,530 photons, 1,632,897 with E>100 MeV.
	theta<66.4 and z<100 selections remove: 4.40 %
	GTI cut removes 0
	Returning tstart=241960000, 1,542,597 photons.
FT2: ft2_2008.fits, MET range 239556977.6-252480028.6, 362996 entries, 73356 (20.2%) in GTI
FT1: P305_Source_003_zmax105.fits, GTI range 244590000.0-247220000.0
	 2,467,921 photons, 1,686,575 with E>100 MeV.
	theta<66.4 and z<100 selections remove: 4.39 %
	GTI cut removes 0
	Returning tstart=244590000, 1,593,093 photons.
FT2: ft2_2008.fits, MET range 239556977.6-252480028.6, 362996 entries, 74096 (20.4%) in GTI
FT1: P305_Source_004_zmax105.fi

In [ ]:
year_sel = lambda month: (month+7)//12
list(map( year_sel, range(12)))

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
ft2[0].header['TSTART']

'239556977.600000'

### Load weekly

In [ ]:
%nbdev_hide
config = Config(verbose=2)
wkfolder = '/home/burnett/weekly'

for wk in [669]: #range(532, 668): #59,100):
    wd = WeeklyData(config, wk, wkfolder)
    wd.save()

/tmp/from_gsfc/week669_ft1.fits exists
/tmp/from_gsfc/week669_ft2.fits exists
GTI times: 638324106 - 638928499 Total: 509592s, fraction 84.3%
Photon (FT1) file /tmp/from_gsfc/week669_ft1.fits: 2,532,645 photons, 2,083,254 with E>100 MeV.
	theta<66.4 and z<100 selections remove: 79.46 %
	GTI cut removes 0
	Returning tstart=638324105, 323,956 photons.
S/C history (FT2) file /tmp/from_gsfc/week669_ft2.fits 17038 entries , 17037 in GTI
Saved to /home/burnett/weekly/week_669.pkl


## check the weekly files

In [ ]:
%nbdev_hide
weekly_folder = Path(wkfolder)
ff = sorted(list(weekly_folder.glob('*.pkl')))
wk = list(map(lambda f: int(os.path.splitext(f)[0][-3:]), ff))
lastweek = pickle.load(open(ff[-1],'rb'))

print(f'Weekly folder "{weekly_folder}" contains {len(wk)} weeks, from {wk[0]} to {wk[-1]}')
print(f'Most recent data to UTC {UTC(MJD(lastweek["tstart"])+7)}')

Weekly folder "/home/burnett/weekly" contains 660 weeks, from 9 to 669
Most recent data to UTC 2021-04-01 00:16


In [ ]:
%nbdev_hide
from nbdev.export import notebook2script
notebook2script()
!date

Converted 00_config.ipynb.
Converted 01_data_man.ipynb.
Converted 01_effective_area.ipynb.
Converted 02_gti.ipynb.
Converted 02_source_data.ipynb.
Converted 03_exposure.ipynb.
Converted 04_photon_data.ipynb.
Converted 05_weights.ipynb.
Converted 06_poisson.ipynb.
Converted 07_cell_data.ipynb.
Converted 07_cells.ipynb.
Converted 08_loglike.ipynb.
Converted 09_lightcurve.ipynb.
Converted 10_simulation.ipynb.
Converted 14_bayesian.ipynb.
Converted index.ipynb.
Tue Apr 13 09:13:32 PDT 2021
